In [1]:
import pandas as pd
df = pd.read_csv('processed_reddit_data.csv')

In [4]:
import pandas as pd
from collections import Counter
import ast
import folium
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

df = pd.read_csv("ner_extracted_locations.csv")

df_places = df.dropna(subset=['place_name'])

all_places = []
for item in df_places['place_name']:
    try:
        evaluated_item = ast.literal_eval(item)
        if isinstance(evaluated_item, list):
            all_places.extend(evaluated_item)
        elif isinstance(evaluated_item, str):
            all_places.append(evaluated_item)
    except (ValueError, SyntaxError):
        cleaned_item = item.strip("[]'")
        if cleaned_item:
            all_places.append(cleaned_item)

geolocator_heatmap = Nominatim(user_agent="all_crisis_locations_heatmap")
all_coordinates = []
for place in all_places:
    try:
        location = geolocator_heatmap.geocode(place, timeout=10)
        if location:
            all_coordinates.append((location.latitude, location.longitude))
        else:
            print(f"Could not geocode for heatmap: {place}")
    except GeocoderTimedOut:
        print(f"Geocoding timed out for heatmap: {place}")
    except Exception as e:
        print(f"An error occurred during geocoding for heatmap ({place}): {e}")

if all_coordinates:
    m_heatmap = folium.Map(location=[20, 0], zoom_start=2)
    HeatMap(all_coordinates).add_to(m_heatmap)
    m_heatmap.save("all_crisis_locations_heatmap.html")
    print("\nHeatmap of all extracted crisis locations saved as crisis_heatmap.html")


Heatmap of all extracted crisis locations saved as crisis_heatmap.html


In [3]:
import pandas as pd
from collections import Counter
import ast
import folium
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

df = pd.read_csv("ner_extracted_locations.csv")

df_places = df.dropna(subset=['place_name'])

all_places = []
for item in df_places['place_name']:
    try:
        evaluated_item = ast.literal_eval(item)
        if isinstance(evaluated_item, list):
            all_places.extend(evaluated_item)
        elif isinstance(evaluated_item, str):
            all_places.append(evaluated_item)
    except (ValueError, SyntaxError):
        cleaned_item = item.strip("[]'")
        if cleaned_item:
            all_places.append(cleaned_item)

location_counts = Counter(all_places)

top_5_locations = location_counts.most_common(5)

print("Top 5 Locations with Highest Crisis Discussions:")
print(top_5_locations)

geolocator = Nominatim(user_agent="top_5_crisis_locations")
top_5_coordinates = []
for place, count in top_5_locations:
    try:
        location = geolocator.geocode(place, timeout=10)
        if location:
            top_5_coordinates.append((location.latitude, location.longitude, count))
        else:
            print(f"Could not geocode: {place}")
    except GeocoderTimedOut:
        print(f"Geocoding timed out for: {place}")
    except Exception as e:
        print(f"An error occurred while geocoding {place}: {e}")

if top_5_coordinates:
    avg_lat = sum(coord[0] for coord in top_5_coordinates) / len(top_5_coordinates)
    avg_lon = sum(coord[1] for coord in top_5_coordinates) / len(top_5_coordinates)
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=3)

    for lat, lon, count in top_5_coordinates:
        folium.Marker(
            [lat, lon],
            popup=f"{geolocator.reverse((lat, lon), exactly_one=True).address}: {count} discussions",
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(m)

    m.save("top_5_crisis_locations_map.html")
    print("\nMap of top 5 crisis locations saved as top_5_crisis_locations_map.html")

Top 5 Locations with Highest Crisis Discussions:
[('canada', 9), ('india', 9), ('texas', 7), ('florida', 6), ('california', 5)]

Map of top 5 crisis locations saved as top_5_crisis_locations_map.html
